In [9]:
#  Setup for C Programming in Colab

import subprocess
import os
import time

# Start session timer
session_start = time.time()

# Check system information
print("\n=== System Information ===")
result = subprocess.run(['uname', '-a'], capture_output=True, text=True)
print(f"System: {result.stdout.strip()}")

result = subprocess.run(['nproc'], capture_output=True, text=True)
print(f"CPU Cores: {result.stdout.strip()}")

result = subprocess.run(['gcc', '--version'], capture_output=True, text=True)
gcc_version = result.stdout.split('\n')[0]
print(f"GCC: {gcc_version}")

# Check for OpenMP support

test_openmp = '''
#include <omp.h>
#include <stdio.h>
int main() {
    printf("OpenMP max threads: %d\\n", omp_get_max_threads());
    return 0;
}
'''

with open('test_openmp.c', 'w') as f:
    f.write(test_openmp)

# Try to compile with OpenMP
compile_result = subprocess.run(['gcc', '-fopenmp', 'test_openmp.c', '-o', 'test_openmp'],
                               capture_output=True, text=True)
if compile_result.returncode == 0:
    run_result = subprocess.run(['./test_openmp'], capture_output=True, text=True)
    print(f"OpenMP: Available - {run_result.stdout.strip()}")
    openmp_available = True
else:
    print("OpenMP: Not available")
    openmp_available = False

print(" Pthreads: Available ")
print("Math library: Available")

# Clean up test files
for file in ['test_openmp.c', 'test_openmp']:
    if os.path.exists(file):
        os.remove(file)

print("\n Environment setup complete!")


=== System Information ===
System: Linux 216ce07b3638 6.1.123+ #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
CPU Cores: 2
GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
OpenMP: Available - OpenMP max threads: 2
 Pthreads: Available 
Math library: Available

 Environment setup complete!


In [13]:
#  the C Source Code


c_source_code = '''#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <string.h>
#include <math.h>
#include <unistd.h>
#include <sys/time.h>
#include <pthread.h>

#ifdef _OPENMP
#include <omp.h>
#endif

// Configuration constants
#define MAX_ARRAY_SIZES 3
#define MAX_ITERATIONS 5
#define MAX_THREADS 8

// Structure to hold performance results
typedef struct {
    char method_name[50];
    double avg_time;
    double std_time;
    double min_time;
    double max_time;
    double speedup;
    double efficiency;
    long long result;
    int array_size;
} PerformanceResult;

// Structure for pthread arguments
typedef struct {
    int *array;
    int start;
    int end;
    long long *partial_sum;
    int thread_id;
} ThreadArgs;

// Global variables
static int num_threads = 4;
static PerformanceResult results[MAX_ARRAY_SIZES * 6]; // 6 methods max
static int result_count = 0;

// Utility functions
double get_time() {
    struct timeval tv;
    gettimeofday(&tv, NULL);
    return tv.tv_sec + tv.tv_usec / 1000000.0;
}

void print_header() {
    printf("=================================================================\\n");
    printf("           PARALLEL ARRAY SUM ANALYSIS )\\n");
    printf("=================================================================\\n");

    #ifdef _OPENMP
    printf("OpenMP: Available (Max threads: %d)\\n", omp_get_max_threads());
    #else
    printf("OpenMP: Not available\\n");
    #endif

    printf("Pthreads: Available\\n");
    printf("System cores: %ld\\n", sysconf(_SC_NPROCESSORS_ONLN));
    printf("=================================================================\\n\\n");
}

// Method 1: Sequential sum
long long sequential_sum(int *array, int size) {
    long long sum = 0;
    for (int i = 0; i < size; i++) {
        sum += array[i];
    }
    return sum;
}

// Method 2: Optimized sequential with loop unrolling
long long optimized_sequential_sum(int *array, int size) {
    long long sum = 0;
    int i;

    // Process 4 elements at a time (loop unrolling)
    for (i = 0; i < size - 3; i += 4) {
        sum += array[i] + array[i+1] + array[i+2] + array[i+3];
    }

    // Handle remaining elements
    for (; i < size; i++) {
        sum += array[i];
    }

    return sum;
}

// Method 3: OpenMP parallel sum
long long openmp_sum(int *array, int size) {
    #ifdef _OPENMP
    long long sum = 0;
    #pragma omp parallel for reduction(+:sum) num_threads(num_threads)
    for (int i = 0; i < size; i++) {
        sum += array[i];
    }
    return sum;
    #else
    printf("  OpenMP not available, using sequential\\n");
    return sequential_sum(array, size);
    #endif
}

// Method 4: Pthread worker function
void* pthread_worker(void* arg) {
    ThreadArgs* args = (ThreadArgs*)arg;
    long long local_sum = 0;

    for (int i = args->start; i < args->end; i++) {
        local_sum += args->array[i];
    }

    args->partial_sum[args->thread_id] = local_sum;
    return NULL;
}

// Method 4: Pthread parallel sum
long long pthread_sum(int *array, int size) {
    pthread_t threads[MAX_THREADS];
    ThreadArgs thread_args[MAX_THREADS];
    long long partial_sums[MAX_THREADS];
    int chunk_size = size / num_threads;

    // Create threads
    for (int i = 0; i < num_threads; i++) {
        thread_args[i].array = array;
        thread_args[i].start = i * chunk_size;
        thread_args[i].end = (i == num_threads - 1) ? size : (i + 1) * chunk_size;
        thread_args[i].partial_sum = partial_sums;
        thread_args[i].thread_id = i;

        if (pthread_create(&threads[i], NULL, pthread_worker, &thread_args[i]) != 0) {
            printf("Error creating thread %d\\n", i);
            return sequential_sum(array, size);
        }
    }

    // Join threads
    for (int i = 0; i < num_threads; i++) {
        pthread_join(threads[i], NULL);
    }

    // Combine results
    long long total_sum = 0;
    for (int i = 0; i < num_threads; i++) {
        total_sum += partial_sums[i];
    }

    return total_sum;
}

// Method 5: SIMD-optimized sum (manual vectorization simulation)
long long simd_sum(int *array, int size) {
    long long sum = 0;
    int vector_size = 4; // Simulate 4-wide SIMD

    // Process vectors of 4 elements
    for (int i = 0; i < size - vector_size + 1; i += vector_size) {
        // Simulate SIMD addition
        long long vector_sum = 0;
        for (int j = 0; j < vector_size; j++) {
            vector_sum += array[i + j];
        }
        sum += vector_sum;
    }

    // Handle remaining elements
    for (int i = (size / vector_size) * vector_size; i < size; i++) {
        sum += array[i];
    }

    return sum;
}

// Method 6: Cache-optimized sum with prefetching simulation
long long cache_optimized_sum(int *array, int size) {
    long long sum = 0;
    const int cache_line_size = 64; // Typical cache line size
    const int ints_per_line = cache_line_size / sizeof(int);

    // Process cache line at a time
    for (int i = 0; i < size; i += ints_per_line) {
        long long line_sum = 0;
        int end = (i + ints_per_line < size) ? i + ints_per_line : size;

        for (int j = i; j < end; j++) {
            line_sum += array[j];
        }
        sum += line_sum;
    }

    return sum;
}

// Performance measurement function
PerformanceResult measure_performance(long long (*method)(int*, int),
                                    int *array, int size,
                                    const char *method_name,
                                    int iterations) {
    PerformanceResult result;
    strcpy(result.method_name, method_name);
    result.array_size = size;

    double times[MAX_ITERATIONS];
    long long results_array[MAX_ITERATIONS];

    printf("  Testing %s...", method_name);
    fflush(stdout);

    // Warm-up run
    method(array, size);

    // Measure iterations
    for (int i = 0; i < iterations; i++) {
        double start_time = get_time();
        results_array[i] = method(array, size);
        double end_time = get_time();
        times[i] = end_time - start_time;
    }

    // Verify consistency
    int consistent = 1;
    for (int i = 1; i < iterations; i++) {
        if (results_array[i] != results_array[0]) {
            consistent = 0;
            break;
        }
    }

    if (consistent) {
        printf(" Success\\n");
    } else {
        printf(" WARNING: Inconsistent results!\\n");
    }

    // Calculate statistics
    result.result = results_array[0];
    result.min_time = times[0];
    result.max_time = times[0];
    result.avg_time = 0;

    for (int i = 0; i < iterations; i++) {
        result.avg_time += times[i];
        if (times[i] < result.min_time) result.min_time = times[i];
        if (times[i] > result.max_time) result.max_time = times[i];
    }
    result.avg_time /= iterations;

    // Calculate standard deviation
    double variance = 0;
    for (int i = 0; i < iterations; i++) {
        variance += (times[i] - result.avg_time) * (times[i] - result.avg_time);
    }
    result.std_time = sqrt(variance / iterations);

    return result;
}

// Generate test array
void generate_test_array(int *array, int size) {
    srand(42); // Fixed seed for reproducibility
    for (int i = 0; i < size; i++) {
        array[i] = (rand() % 100) + 1; // Random integers 1-100
    }
}

// Calculate expected sum for verification
long long calculate_expected_sum(int *array, int size) {
    long long sum = 0;
    for (int i = 0; i < size; i++) {
        sum += array[i];
    }
    return sum;
}

// Run comparison for a specific array size
void run_comparison(int array_size, int iterations) {
    printf("\\n============================================================\\n");
    printf("TESTING ARRAY SIZE: %d elements\\n", array_size);
    printf("Threads: %d | Iterations: %d\\n", num_threads, iterations);
    printf("============================================================\\n");

    // Allocate and generate test array
    printf("Generating test data...\\n");
    int *array = (int*)malloc(array_size * sizeof(int));
    if (!array) {
        printf("Error: Could not allocate memory for array\\n");
        return;
    }

    generate_test_array(array, array_size);
    long long expected_sum = calculate_expected_sum(array, array_size);
    printf("✓ Array generated. Expected sum: %lld\\n", expected_sum);

    printf("\\nRunning performance tests...\\n");

    // Define methods to test
    struct {
        long long (*func)(int*, int);
        const char* name;
    } methods[] = {
        {sequential_sum, "Sequential"},
        {optimized_sequential_sum, "Optimized Sequential"},
        {openmp_sum, "OpenMP Parallel"},
        {pthread_sum, "Pthread Parallel"},
        {simd_sum, "SIMD Optimized"},
        {cache_optimized_sum, "Cache Optimized"}
    };

    int num_methods = sizeof(methods) / sizeof(methods[0]);
    PerformanceResult method_results[6];
    double baseline_time = 0;

    // Test each method
    for (int i = 0; i < num_methods; i++) {
        method_results[i] = measure_performance(methods[i].func, array,
                                              array_size, methods[i].name,
                                              iterations);

        // Set baseline time (first method)
        if (i == 0) {
            baseline_time = method_results[i].avg_time;
        }

        // Calculate speedup and efficiency
        method_results[i].speedup = baseline_time / method_results[i].avg_time;
        method_results[i].efficiency = method_results[i].speedup / num_threads;

        // Store in global results
        results[result_count++] = method_results[i];
    }

    // Display results table
    printf("\\nPERFORMANCE RESULTS:\\n");
    printf("%-22s | %-12s | %-8s | %-10s | %s\\n",
           "Method", "Time (s)", "Speedup", "Efficiency", "Status");
    printf("--------------------------------------------------------------------------\\n");

    for (int i = 0; i < num_methods; i++) {
        const char* status = (method_results[i].result == expected_sum) ? "Correct" : "Wrong";
        printf("%-22s | %12.6f | %8.2fx | %10.2f | %s\\n",
               method_results[i].method_name,
               method_results[i].avg_time,
               method_results[i].speedup,
               method_results[i].efficiency,
               status);
    }

    free(array);
}

// Generate summary report
void generate_summary_report() {
    if (result_count == 0) {
        printf("No results available for summary!\\n");
        return;
    }

    printf("\\n================================================================================\\n");
    printf("                    COMPREHENSIVE PERFORMANCE REPORT\\n");
    printf("================================================================================\\n");
    printf("Total Tests: %d\\n", result_count);
    printf("CPU Cores: %ld\\n", sysconf(_SC_NPROCESSORS_ONLN));

    // Find best performing method overall
    double best_speedup = 0;
    char best_method[50];

    for (int i = 0; i < result_count; i++) {
        if (results[i].speedup > best_speedup) {
            best_speedup = results[i].speedup;
            strcpy(best_method, results[i].method_name);
        }
    }

    printf("\\nBEST PERFORMING METHOD: %s (%.2fx speedup)\\n", best_method, best_speedup);



    printf("\\nAnalysis complete !\\n");
}

// Main function
int main(int argc, char *argv[]) {
    print_header();

    // Set number of threads based on system or command line
    if (argc > 1) {
        num_threads = atoi(argv[1]);
        if (num_threads <= 0 || num_threads > MAX_THREADS) {
            num_threads = 2; // Default for Colab
        }
    } else {
        long cores = sysconf(_SC_NPROCESSORS_ONLN);
        num_threads = (cores > MAX_THREADS) ? MAX_THREADS : (int)cores;
    }

    printf("Using %d threads for parallel operations\\n", num_threads);

    // Test array sizes (optimized for Colab)
    int array_sizes[] = {500000, 1000000, 2000000};
    int iterations = 3; // Reduced for faster execution in Colab

    printf("Testing with array sizes: ");
    for (int i = 0; i < MAX_ARRAY_SIZES; i++) {
        printf("%d ", array_sizes[i]);
    }
    printf("\\n");

    // Run comparisons for each array size
    for (int i = 0; i < MAX_ARRAY_SIZES; i++) {
        run_comparison(array_sizes[i], iterations);
    }

    // Generate final report
    printf("\\nGenerating final report...\\n");
    generate_summary_report();

    return 0;
}'''

# Write the C source code to file
with open('array_sum_analysis.c', 'w') as f:
    f.write(c_source_code)

print("✓ C source code created: array_sum_analysis.c")
print(f"✓ Source code size: {len(c_source_code)} characters")
print("✓ Ready for compilation!")

✓ C source code created: array_sum_analysis.c
✓ Source code size: 12423 characters
✓ Ready for compilation!


In [14]:
# Compile and Execute the C Program

import subprocess
import os

# Compilation options
compile_commands = [
    {
        'name': 'Basic (Sequential only)',
        'cmd': ['gcc', '-Wall', '-O3', '-o', 'array_sum_basic', 'array_sum_analysis.c', '-lm', '-lpthread'],
        'executable': 'array_sum_basic'
    },
    {
        'name': 'Full (OpenMP + Pthreads)',
        'cmd': ['gcc', '-Wall', '-O3', '-fopenmp', '-D_OPENMP', '-o', 'array_sum_full', 'array_sum_analysis.c', '-lm', '-lpthread'],
        'executable': 'array_sum_full'
    }
]

successful_builds = []

for build in compile_commands:
    print(f"\nCompiling {build['name']}...")
    print(f"Command: {' '.join(build['cmd'])}")

    result = subprocess.run(build['cmd'], capture_output=True, text=True)

    if result.returncode == 0:
        print(f" Successfully compiled: {build['executable']}")
        successful_builds.append(build)

        # Check file size
        if os.path.exists(build['executable']):
            size = os.path.getsize(build['executable'])
            print(f"  Executable size: {size} bytes")
    else:
        print(f" Compilation failed for {build['name']}")
        if result.stderr:
            print(f"  Error: {result.stderr}")

if successful_builds:
    print(f"\n Successfully compiled {len(successful_builds)} version(s)")
    selected_build = successful_builds[-1]  # Use the most feature-rich version
    print(f" Selected for execution: {selected_build['executable']}")

    # Execute the program
    executable = selected_build['executable']

    # Get system info for optimal thread count
    cores_result = subprocess.run(['nproc'], capture_output=True, text=True)
    cores = int(cores_result.stdout.strip())
    optimal_threads = min(cores, 4)  # Limit for Colab performance

    print(f"\nSystem cores: {cores}")
    print(f"Using threads: {optimal_threads}")
    print(f"Executable: {executable}")
    print("\n" + "="*60)

    # Execute the program
    start_time = time.time()

    try:
        result = subprocess.run([f'./{executable}', str(optimal_threads)],
                              capture_output=True, text=True, timeout=120)


        if result.returncode == 0:
            print(result.stdout)
            print(f"\n" + "="*60)
            print(f"✓ Execution completed successfully!")

        else:
            print(f"✗ Execution failed with return code: {result.returncode}")
            if result.stderr:
                print(f"Error output: {result.stderr}")
            if result.stdout:
                print(f"Standard output: {result.stdout}")

    except Exception as e:
        print(f" Execution error: {e}")

else:
    print(" No executable available to run")
    print("Please check the compilation step above")


Compiling Basic (Sequential only)...
Command: gcc -Wall -O3 -o array_sum_basic array_sum_analysis.c -lm -lpthread
 Successfully compiled: array_sum_basic
  Executable size: 25576 bytes

Compiling Full (OpenMP + Pthreads)...
Command: gcc -Wall -O3 -fopenmp -D_OPENMP -o array_sum_full array_sum_analysis.c -lm -lpthread
 Successfully compiled: array_sum_full
  Executable size: 26056 bytes

 Successfully compiled 2 version(s)
 Selected for execution: array_sum_full

System cores: 2
Using threads: 2
Executable: array_sum_full

           PARALLEL ARRAY SUM ANALYSIS )
OpenMP: Available (Max threads: 2)
Pthreads: Available
System cores: 2

Using 2 threads for parallel operations
Testing with array sizes: 500000 1000000 2000000 

TESTING ARRAY SIZE: 500000 elements
Threads: 2 | Iterations: 3
Generating test data...
✓ Array generated. Expected sum: 25223450

Running performance tests...
  Testing Sequential... Success
  Testing Optimized Sequential... Success
  Testing OpenMP Parallel... Succe